In [ ]:
#to import from google drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import PIL
from PIL import Image
import cv2
from skimage.transform import hough_line, hough_line_peaks
from skimage.feature import canny
from skimage.draw import line as draw_line
from skimage import data
from skimage.transform import probabilistic_hough_line
from matplotlib import cm


#defining some functions to read in the data, ripping these from not so simple pendulum
def read_vid(video):
    imagearray = np.array(video.read())[1]
    return imagearray

def crop_image(imagearray, startx, endx, starty, endy):
    return imagearray[starty:endy,startx:endx,:]

#split video into frames
filename = "1"
mp4name = filename + ".mp4"
print(mp4name)
video = cv2.VideoCapture(mp4name)

framecount = 0
while np.array(video.read())[0]:

    framecount += 1

video = cv2.VideoCapture(mp4name)
print(framecount)

In [ ]:
def auto_crop(imagearray):
    imagearray = crop_image(imagearray, 500, 3500, 200, 2000)
    xmaxpos = 0


    for y,i in enumerate(imagearray):
        for x,r in enumerate(i):
            if r[2] > 100 and r[0] <= 80 and x > xmaxpos:
                xmaxpos = x

    adjust = 100#this is to prevent it catching the first turbulant section of the wave
    print(xmaxpos) #this also provides position data for the boat

    cropval = xmaxpos + adjust
    imagearray = crop_image(imagearray, cropval, len(imagearray[0]), 0, len(imagearray))
    plt.imshow(imagearray)

    return imagearray, xmaxpos

In [ ]:
def image_processing(imagearray):
    copy = np.array(imagearray)
    copy = copy.flatten()

    #going to build my own filter
    #mean brightness and std
    mean = np.mean(copy)
    std = np.std(copy)



    minvalue = int(mean + 1.9*std)

    lower = np.array([minvalue, minvalue, minvalue])
    upper = np.array([255, 255, 255])
    mask = cv2.inRange(np.array(imagearray), lower, upper)

    #de noising
    blur = cv2.GaussianBlur(mask, (0,0), sigmaX=180, sigmaY=180)
    denoised = cv2.divide(mask, blur, scale=250)
    denoised = cv2.GaussianBlur(mask, (0,0), sigmaX=5, sigmaY=5)

    mask = cv2.inRange(denoised, minvalue, 255)

    output = denoised
    return output

In [ ]:
def find_lines(imagearray):
    #image processing first
    processed = image_processing(imagearray)

    lines = []
    lines = probabilistic_hough_line(processed, threshold=400, line_length=350, line_gap=50)

    #remove obviously bad values
    badvals = []

    angles = []
    try:
        for count,line in enumerate(lines):
            p0, p1 = line
            dy = p1[1] - p0[1] # this might be wrong
            dx = p1[0] - p0[0]
            angle = np.arctan2(dy,dx)


            if np.abs(angle) >= 1.2 or np.abs(angle) <= 0.2:

                badvals.append(count)
            else:
                angles.append(np.abs(angle))

        lines = np.array(lines)
        lines = np.delete(lines,badvals,0)#this drops all the bad lines, but will not prevent them from being generated, band aid solution

        plt.imshow(imagearray)
        for line in lines:
            p0, p1 = line
            plt.plot((p0[0], p1[0]), (p0[1], p1[1]))

        plt.show()


        print("wavelength search began")
        for line1 in lines:
            p10, p11 = line1
            #need to generate a line from this line

            dy1 = p11[1] - p10[1]


            dx1 = p11[0] - p10[0]

            m1 = dy1/dx1
            c1 = p10[1] - p1[0] * m1
            for line2 in lines:
                p20, p21 = line2
                #generate another line from this line
                dy2 = p21[1] - p20[1]
                dx2 = p21[0] - p20[0]
                m2 = dy2/dx2
                c2 = p20[1] - p20[0] * m2
                #compare the angles


                if m1 <= 0: ##because its negative most the time
                    lower = m1 * 0.9
                    upper = m1 * 1.1

                    if m2 <= lower and m2 >= upper:
                        num = np.abs(c1-c2)
                        denom = np.sqrt(1+((m1+m2)/2)**2)

                        d = num/denom

                        if d > 50:
                            print("Wavelength found: ", d)
                            wavelengths.append(d)


        return angles

    except:
        print("No lines found")

In [ ]:
import pickle

#generate the file names
filenames = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31", "32", "33", "34"]

extension = ".mov"
mp4names = []
for i in filenames:
    temp = i + extension
    mp4names.append(temp)


#loop process for all files
for count,i in enumerate(mp4names):
    print(i)

    wavelengths = []
    allangles = []
    positions = []


    video = cv2.VideoCapture(i)

    framecount = 0
    while np.array(video.read())[0]:

        framecount += 1

    video = cv2.VideoCapture(i)


    #string it all together for all the frames
    for _ in range(0,framecount):

        imagearray= np.array(video.read())[1]

        imagearray, pos = auto_crop(imagearray)#taking image and location from the crop

        positions.append(pos)

        angles = find_lines(imagearray)

        allangles.append(angles)

    #save angles
    with open(filenames[count], 'wb+') as file:
        allangles = np.array(allangles)
        pickle.dump(allangles, file)
        #print(allangles)
        file.close()

    #save velocity
    name = filenames[count] + "vel"
    #print(positions)
    with open(name, 'wb+') as file:
        positions = np.array(positions)
        pickle.dump(positions, file)
        file.close()

    name = filenames[count] + "wav"
    with open(name, 'wb+') as file:
        positions = np.array(wavelengths)
        pickle.dump(positions, file)
        file.close()